In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf

# -------------------------
# Dataset (validation set)
# -------------------------
img_size = (224, 224)
batch_size = 32
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

# -------------------------
# Load your saved models
# -------------------------
customcnn_loaded     = tf.keras.models.load_model(r'D:\sign_language_project_updated\saved_models\CustomCNN_best.h5')
alexnet_loaded       = tf.keras.models.load_model(r'D:\sign_language_project_updated\saved_models\AlexNet_best.h5')
efficientnet_loaded  = tf.keras.models.load_model(r'D:\sign_language_project_updated\saved_models\EfficientNetB0_best.h5')
resnet50_loaded      = tf.keras.models.load_model(r'D:\sign_language_project_updated\saved_models\ResNet50_best.h5')
mobilenetv2_loaded   = tf.keras.models.load_model(r'D:\sign_language_project_updated\saved_models\MobileNet_best.h5')

models = {
    "CustomCNN": customcnn_loaded,
    "AlexNet": alexnet_loaded,
    "EfficientNet": efficientnet_loaded,
    "ResNet50": resnet50_loaded,
    "MobileNetV2": mobilenetv2_loaded,
}

# -------------------------
# Evaluation function
# -------------------------
def evaluate_model(model, val_ds, model_name, class_names):
    # Get ground truth
    y_true = np.concatenate([labels.numpy() for _, labels in val_ds], axis=0)

    # Get predictions in one pass
    y_pred_probs = model.predict(val_ds, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    report = classification_report(
        y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0
    )
    accuracy = accuracy_score(y_true, y_pred)

    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    }

# -------------------------
# Run evaluation on all models (CPU forced)
# -------------------------
results = []
with tf.device("/CPU:0"):   # ensures inference runs on CPU
    for name, model in models.items():
        results.append(evaluate_model(model, val_ds, name, class_names))

df_results = pd.DataFrame(results)
print(df_results)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf

# -------------------------
# Dataset (validation set)
# -------------------------
img_size = (224, 224)
batch_size = 32
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

# -------------------------
# Load your saved models
# -------------------------

models = {
    "CustomCNN": customcnn_loaded,
    "AlexNet": alexnet_loaded,
    "EfficientNet": efficientnet_loaded,
    "ResNet50": resnet50_loaded,
    "MobileNetV2": mobilenetv2_loaded,
}

# -------------------------
# Evaluation function
# -------------------------
def evaluate_model(model, val_ds, model_name, class_names):
    # Get ground truth
    y_true = np.concatenate([labels.numpy() for _, labels in val_ds], axis=0)

    # Get predictions in one pass
    y_pred_probs = model.predict(val_ds, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    report = classification_report(
        y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0
    )
    accuracy = accuracy_score(y_true, y_pred)

    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    }

# -------------------------
# Run evaluation on all models (CPU forced)
# -------------------------
results = []
with tf.device("/CPU:0"):   # ensures inference runs on CPU
    for name, model in models.items():
        results.append(evaluate_model(model, val_ds, name, class_names))

df_results = pd.DataFrame(results)

print(df_results)

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

def inference_table(models_dict, val_ds, class_names, num_samples=5):
    # Take random samples from validation set
    sample_data = list(val_ds.unbatch().take(num_samples))

    num_models = len(models_dict)
    plt.figure(figsize=(5 * (num_models+1), num_samples * 3))

    for i, (img, label) in enumerate(sample_data):
        # Convert tensor to numpy and bring back to [0,255]
        img_np = (img.numpy() * 255).astype("uint8")
        gt_label = class_names[int(label)]

        # First column = ground truth
        plt.subplot(num_samples, num_models+1, i*(num_models+1) + 1)
        plt.imshow(img_np)
        plt.title(f"Ground Truth:\n{gt_label}", color="blue")
        plt.axis("off")

        # Predictions for each model
        for j, (name, model) in enumerate(models_dict.items()):
            pred = model.predict(tf.expand_dims(img, axis=0), verbose=0)
            pred_class = np.argmax(pred, axis=1)[0]
            pred_label = class_names[pred_class]

            # Green if correct, red if wrong
            color = "green" if pred_label == gt_label else "red"

            plt.subplot(num_samples, num_models+1, i*(num_models+1) + j + 2)
            plt.imshow(img_np)
            plt.title(f"{name}\nGT: {gt_label}\nPred: {pred_label}", color=color)
            plt.axis("off")

    plt.tight_layout()
    plt.show()

# Dictionary of models to evaluate
all_models = {
    "CustomCNN": custom_cnn_trained,
    "AlexNet": alexnet_trained,
    "ResNet50": resnet50_trained,
    "EfficientNet": efficientnet_trained,
    "MobileNet": mobilenet_trained
}

# Run table visualization
inference_table(all_models, val_ds, class_names, num_samples=5)